In [2]:
import keras.optimizers
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from keras.models import Model

In [7]:
data = tf.keras.utils.image_dataset_from_directory('data', image_size=(224,224), batch_size=16, label_mode='categorical')
data_iterator = data.as_numpy_iterator()

Found 14100 files belonging to 7 classes.


ValueError: 'size' must be a 1-D Tensor of 2 elements: new_height, new_width

In [4]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [5]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [8]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False,input_shape=(224, 224, 3))


In [24]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Flatten()(x)  # Flatten the output of the VGG16 base model
x = Dense(units=7, activation='softmax')(x)  # Add a Dense layer with softmax activation



In [25]:
model = Model(inputs=base_model.input, outputs=x)


In [29]:
for layer in base_model.layers:
    layer.trainable = False

In [30]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [31]:
model.fit(train, validation_data=val, epochs=5)

Epoch 1/5
617/617 [==============================] - 81s 120ms/step - loss: 0.7765 - accuracy: 0.6796 - val_loss: 0.8176 - val_accuracy: 0.6786
Epoch 2/5
617/617 [==============================] - 73s 118ms/step - loss: 0.6451 - accuracy: 0.7270 - val_loss: 1.0130 - val_accuracy: 0.6626
Epoch 3/5
617/617 [==============================] - 75s 122ms/step - loss: 0.5836 - accuracy: 0.7598 - val_loss: 1.1981 - val_accuracy: 0.6076
Epoch 4/5
617/617 [==============================] - 83s 135ms/step - loss: 0.5510 - accuracy: 0.7717 - val_loss: 0.6224 - val_accuracy: 0.7663
Epoch 5/5
166/617 [=======>......................] - ETA: 47s - loss: 0.5055 - accuracy: 0.7948

KeyboardInterrupt: 